In [ ]:
import scanpy as sc
import numpy as np
import seaborn as sns
import squidpy as sq
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Rectangle

In [ ]:
basepath = '/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/figures/'
dpi = 300
region_cmap = {str(i):sc.pl.palettes.default_102[i] for i in range(40)}
techs = ['starmap', 'slideseq', 'visium']
figsizes = [(5,6), (6,6), (5,6)]
ss = {'starmap':15, 'slideseq':12, 'visium':100}

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_xtech_spin_recolored.h5ad')

In [ ]:
adata.obs['region']

In [ ]:
sc.set_figure_params(figsize=(5,6))
sc.pl.embedding(adata[adata.obs['tech']=='starmap'], basis='spatial', color='region', s=10)

In [ ]:
sc.set_figure_params(figsize=(5,6))
sc.pl.embedding(adata[adata.obs['tech']=='starmap'], basis='spatial', color='region', s=10, groups='FC')

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata[adata.obs['tech']=='slideseq'], basis='spatial', color='region', s=10, groups='FC')

In [ ]:
sc.set_figure_params(figsize=(5,6))
sc.pl.embedding(adata[adata.obs['tech']=='visium'], basis='spatial', color='region', s=50, groups='FC')

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='region')
sc.pl.rank_genes_groups_dotplot(adata, groupby='region', groups=['FC'], n_genes=5)

# Preprocess again

## Remove bad STARmap genes

In [ ]:
adata1 = sc.read_h5ad('/stanley/WangLab/kamal/data/marmoset/mpfc/mpfc.h5ad')
bad_genes = {'CALB1', 'OXTR', 'FOXP2', 'FIGN', 'SORCS1', 'SPP1', 'RGS4'}
all_genes = {gene for gene in adata1.var_names}
good_genes = list(all_genes - bad_genes)
adata1 = adata1[:,good_genes]
adata1.obsm['spatial'] = adata1.obs[['x','y']].values
adata1.var_names = [var[0].upper()+var[1:].lower() for var in adata1.var_names]
adata1.raw = adata1.copy()

## Combine into one dataset and normalize

In [ ]:
adata2 = sq.datasets.visium_hne_adata()
adata2.obsm['spatial'][:,1] = -adata2.obsm['spatial'][:,1]
adata2.X = np.array(adata2.raw[:,adata2.var_names].X.todense()) # start from raw to guarantee same pp

In [ ]:
adata1.obs['species'] = ['marmoset' for _ in range(len(adata1))]
adata2.obs['species'] = ['mouse' for _ in range(len(adata2))]
adata = sc.concat([adata1, adata2])

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.scale(adata)

In [ ]:
adata.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/mpfc_marmoset_mouse.h5ad')

In [ ]:
X = adata[:,marker].X.copy()
vmin = X.min()
vmax = X.max()
species = ['marmoset', 'mouse']
sc.set_figure_params(figsize=(5,6))
ss = [10, 80]
marker = 'Lamp5'
print(marker)
for i in range(len(species)):
    subdata = adata[adata.obs['species']==species[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[i], title='', colorbar_loc=None, return_fig=True, vmin=vmin, vmax=vmax)
    plt.axis('off')

# Preprocess data

In [ ]:
adata_starmap = sc.read_h5ad('/stanley/WangLab/kamal/data/mouse/hpc/hpc_preprocessed.h5ad')
adata_starmap.obsm['spatial'] = adata_starmap.obs[['x','y']].values
adata_starmap.var_names = [name[0]+name[1:].lower() for name in adata_starmap.var_names]

adata_slideseq = sq.datasets.slideseqv2()
adata_slideseq.obsm['spatial'][:,1] = -adata_slideseq.obsm['spatial'][:,1]
adata_slideseq.X = np.array(adata_slideseq.raw[:,adata_slideseq.var_names].X.todense()) # start from raw to guarantee same pp
sc.pp.normalize_total(adata_slideseq)
sc.pp.log1p(adata_slideseq)
sc.pp.scale(adata_slideseq)

adata_visium = sq.datasets.visium_hne_adata()
adata_visium.obsm['spatial'][:,1] = -adata_visium.obsm['spatial'][:,1]
adata_visium.X = np.array(adata_visium.raw[:,adata_visium.var_names].X.todense()) # start from raw to guarantee same pp
sc.pp.normalize_total(adata_visium)
sc.pp.log1p(adata_visium)
sc.pp.scale(adata_visium)

adatas = [adata_starmap, adata_slideseq, adata_visium]

In [ ]:
sc.set_figure_params(figsize=(6,6))
for i in range(len(adatas)):
    sc.pl.embedding(adatas[i], basis='spatial', color='Lamp5', s=ss[techs[i]])

In [ ]:
for i in range(len(adatas)):
    adatas[i].write(f'/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_{techs[i]}.h5ad')

In [ ]:
for i in range(len(techs)):
    adata = sc.read_h5ad(f'/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_{techs[i]}.h5ad')
    print(adata)

Pass to SPIN on cluster

# Tissue and latent colored by region

In [ ]:
pooldata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_xtech_spin_num_nbrs_array_30_400_10.h5ad')

In [ ]:
pooldata

In [ ]:
res = 0.45
sc.tl.leiden(pooldata, resolution=res, neighbors_key='region', key_added='region')

In [ ]:
pooldata.obs['region']

In [ ]:
for i in range(len(techs)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = pooldata[pooldata.obs['tech']==techs[i]]
    sc.pl.embedding(subdata, basis='spatial', color='region', palette=region_cmap, s=ss[techs[i]], title='', legend_loc=None, return_fig=True)
    plt.axis('off')
    # figname = f'{techs[i]}_tissue_colored_by_region.png'
    # savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
for i in range(len(techs)):
    subdata = pooldata[pooldata.obs['tech']==techs[i]]
    sc.set_figure_params(figsize=(5,5))
    sc.pl.embedding(subdata, basis='X_umap_spin', color='region', palette=region_cmap, s=8, title='', legend_loc=None, return_fig=True)
    plt.axis('off')
    # figname = f'{techs[i]}_latent_colored_by_region.png'
    # savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# pooldata.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_xtech_spin_num_nbrs_array_30_200_20_res1.h5ad')

# Recolor

In [ ]:
adata = pooldata[pooldata.obs['tech']=='starmap']
sc.pl.embedding(adata, basis='spatial', color='region', s=20, legend_loc='on data', title='', palette=region_cmap)

In [ ]:
sc.pl.embedding(adata, basis='spatial', groups=['23'], color='region', s=20, legend_loc='on data', title='', palette=region_cmap)

In [ ]:
adata = pooldata[pooldata.obs['tech']=='slideseq']
sc.pl.embedding(adata, basis='spatial', groups=['23'], color='region', s=20, legend_loc='on data', title='', palette=region_cmap)

In [ ]:
leiden2region = {
    '0': 'TH1',
    '1': 'fiber tract',
    '2': 'L6',
    '3': 'L2/3',
    '4': 'DG',
    '5': 'HY',
    '6': 'CA1',
    '7': 'L5',
    '8': 'RSP',
    '9': 'CP/BLA',
    '10': 'L4',
    '11': 'MEA/COA',
    '12': 'CA2/3',
    '13': 'meninges/L1',
    '14': 'HPFmo',
    '15': 'PIR',
    '16': 'TH2',
    '17': 'MHb',
    '18': 'L6b',
    '19': 'chpl',
    '20': 'LHb',
    '21': 'RT',
    '22': 'VL',
    '23': 'FC',

}
pooldata.obs['leiden'] = pooldata.obs['region'].copy()
pooldata.obs['region'] = pooldata.obs['leiden'].map(leiden2region)
pooldata.obs['region'] = pooldata.obs['region'].astype('category')

## Use prior cmap

In [ ]:
olddata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/spatial_autocorrelation/adata_all_techniques.h5ad')
old_regions = olddata.obs['region_smooth_subsampled'].unique()
palette = sc.pl.palettes.default_102
old_cmap = {old_regions[i]:palette[i] for i in range(len(old_regions))}

In [ ]:
old_regions = set(old_regions)
new_regions = set(pooldata.obs['region'].unique())
shared_regions = old_regions.intersection(new_regions)
all_regions = old_regions.union(new_regions)

new_cmap = dict()
for region in shared_regions:
    new_cmap[region] = old_cmap[region]

print(all_regions-shared_regions)

In [ ]:
old_regions = set(old_regions)
new_regions = set(pooldata.obs['region'].unique())
shared_regions = old_regions.intersection(new_regions)
all_regions = old_regions.union(new_regions)

new_cmap = dict()
for region in shared_regions:
    new_cmap[region] = old_cmap[region]

print(all_regions-shared_regions)

In [ ]:
new_cmap['TH1'] = old_cmap['TH']
new_cmap['chpl'] = old_cmap['chpl1']
new_cmap['L6b'] = old_cmap['BLA']
new_cmap['FC'] = palette[len(new_regions)]
new_cmap['TH2'] = old_cmap['chpl2']
new_cmap['CP/BLA'] = old_cmap['CP']

In [ ]:
# Verify uniqueness of colors
len(new_cmap.values()) == len(set(new_cmap.values()))

In [ ]:
region_cmap = new_cmap

In [ ]:
ss = {'starmap':20, 'slideseq':20, 'visium':120}

In [ ]:
for i in range(len(techs)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = pooldata[pooldata.obs['tech']==techs[i]]
    sc.pl.embedding(subdata, basis='spatial', color='region', palette=region_cmap, s=ss[techs[i]], title='', legend_loc=None, return_fig=True)
    plt.axis('off')
    figname = f'{techs[i]}_tissue_colored_by_region.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
for i in range(len(techs)):
    subdata = pooldata[pooldata.obs['tech']==techs[i]]
    sc.set_figure_params(figsize=(5,5))
    sc.pl.embedding(subdata, basis='X_umap_spin', color='region', palette=region_cmap, s=8, title='', legend_loc=None, return_fig=True)
    plt.axis('off')
    figname = f'{techs[i]}_latent_colored_by_region.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(pooldata, basis='X_umap_spin', color='region', palette=region_cmap, s=8, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = 'all_latent_colored_by_region.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(pooldata, basis='X_umap_spin', color='tech', s=15, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = 'all_latent_colored_by_tech.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# Non-shuffled (above) actually looks better
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(adata)), size=len(adata), replace=False)
sc.pl.embedding(pooldata[random_idxs], basis='X_umap_spin', color='tech', s=10, title='', legend_loc=None, return_fig=True)
plt.axis('off')
# figname = 'all_latent_colored_by_tech.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
adata = pooldata[pooldata.obs['tech']=='starmap']
sc.pl.embedding(adata, basis='X_umap_spin', color='region', s=20, legend_loc='on data', title='', palette=region_cmap)

In [ ]:
pooldata.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_xtech_spin_recolored.h5ad')

# Tissue zooms colored by region

In [ ]:
def get_zoom(adata, x, y, width, height):
    zoomdata = adata[adata.obsm['spatial'][:,0]>x]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,0]<x+width]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]>y]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]<y+height]
    return zoomdata

In [ ]:
zoomdata_starmap = get_zoom(pooldata[pooldata.obs['tech']=='starmap'], 11000, 17500, 5000, 8500)
zoomdata_visium = get_zoom(pooldata[pooldata.obs['tech']=='visium'], 5000, -7000, 2000, 3000)

In [ ]:
sc.set_figure_params(figsize=(5,6))
zoomtechs = ['starmap', 'visium']
zoomdatas = [zoomdata_starmap, zoomdata_visium]
zoomss = [150, 700]
for i in range(len(zoomtechs)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color='region', palette=region_cmap, s=zoomss[i], return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
    figname = f'zoom_tissue_{zoomtechs[i]}_colored_by_region.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

# Cross-species analysis

## Full mouse slice

Load output from SPIN on the cluster

In [ ]:
species = ['marmoset', 'mouse']
figsizes = [(2,6), (5,6)]
ss = [5, 80]

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/data/integrated/spatial/marmoset_mouse/marmoset_mouse_xtech_spin_goodgenes_batchcorr_20pcs.h5ad')

In [ ]:
sc.tl.leiden(adata, neighbors_key='region', key_added='region', resolution=0.3)

In [ ]:
for i in range(len(species)):
    subdata = adata[adata.obs['species']==species[i]]
    sc.set_figure_params(figsize=figsizes[i])
    sc.pl.embedding(subdata, basis='spatial', color='region', s=ss[i], return_fig=True, frameon=False, title='', edgecolor=None)
    # figname = f'tissue_{species[i]}_colored_by_region.png'
    # savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
for i in range(len(species)):
    subdata = adata[adata.obs['species']==species[i]]
    sc.set_figure_params(figsize=(5,5))
    sc.pl.embedding(subdata, basis='X_umap_spin', color='region', s=10, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
    # figname = f'tissue_{species[i]}_colored_by_region.png'
    # savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='region', s=10, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
# figname = f'tissue_{species[i]}_colored_by_region.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='species', s=10, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
# figname = f'tissue_{species[i]}_colored_by_region.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

## Mouse cortex only

### Load output from cross tech analysis on cluster

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_xtech_spin_recolored.h5ad')

In [ ]:
adata = adata[adata.obs['tech']=='visium']

In [ ]:
sc.pl.embedding(adata, basis='spatial', color='region', palette=sc.pl.palettes.default_102, s=100)

In [ ]:
# ctx_regions = ['L2/3', 'L4', 'L5', 'L6', 'L6b', 'fiber tract']
ctx_regions = ['L2/3', 'L4', 'L5', 'L6', 'L6b']
sc.pl.embedding(adata, basis='spatial', color='region', groups=ctx_regions, palette=sc.pl.palettes.default_102)

In [ ]:
adata = adata[np.isin(adata.obs['region'],ctx_regions)]

In [ ]:
adata.obs

In [ ]:
adata.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_visium_ctx.h5ad')

In [ ]:
fulldata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_visium.h5ad')

In [ ]:
fulldata

In [ ]:
ctxidxs = list(adata.obs.index)
fullidxs = list(fulldata.obs.index)

In [ ]:
idxs = [fullidxs.index(ctxidxs[i]) for i in range(len(ctxidxs))]

In [ ]:
newdata = fulldata[idxs]

In [ ]:
sc.pl.embedding(newdata, basis='spatial', s=100)

In [ ]:
newdata.X

In [ ]:
newdata

In [ ]:
adata

In [ ]:
newdata.var

In [ ]:
sc.pp.filter_genes(newdata, min_cells=10)

In [ ]:
newdata

In [ ]:
newdata.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_visium_ctx.h5ad')

### Run SPIN on cluster to integrate the above ctx portion of visium w starmap marmoset

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/data/integrated/spatial/marmoset_mouse/marmoset_mouse_xtech_spin_goodgenes_batchcorr_10pcs_ctx.h5ad')

In [ ]:
adata

In [ ]:
sc.tl.leiden(adata, neighbors_key='region', key_added='region', resolution=0.42)

In [ ]:
subdata = adata[adata.obs['species']=='marmoset']
sc.set_figure_params(figsize=(2,6))
sc.pl.embedding(subdata, basis='spatial', color='region', s=5, return_fig=True, frameon=False, title='', edgecolor=None)
plt.show()

In [ ]:
subdata = adata[adata.obs['species']=='mouse']
sc.set_figure_params(figsize=(6,6))
sc.pl.embedding(subdata, basis='spatial', color='region', s=100, return_fig=True, frameon=False, title='', edgecolor=None)
plt.show()

In [ ]:
adata_full = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_xtech_spin_num_nbrs_array_30_200_20_res1.h5ad')
adata_full = adata_full[adata_full.obs['tech']=='visium']

In [ ]:
plt.figure(figsize=(5,6))
plt.scatter(*adata_full.obsm['spatial'].T, color=[0.5]*3, s=30)

subdata = adata[adata.obs['species']=='mouse']
sc.set_figure_params(figsize=(5,6))
sc.pl.embedding(subdata, basis='spatial', color='region', s=80, frameon=False, title='', edgecolor=None, ax=plt.gca())
plt.show()

In [ ]:
adata[adata.obs['species']=='marmoset'].obs['region'].value_counts()

In [ ]:
# Filter out regions with fewer than 500 cells in marmoset
adata = adata[adata.obs['region']!='6']

In [ ]:
leiden2region = {
    '0': 'L2/3',
    '1': 'L5',
    '2': 'L4',
    '3': 'L6',
    '4': 'fiber tract',
    '5': 'meninges/L1',
    '6': 'L6b',
}
adata.obs['leiden'] = adata.obs['region'].copy()
adata.obs['region'] = adata.obs['leiden'].map(leiden2region)
adata.obs['region'] = adata.obs['region'].astype('category')

In [ ]:
olddata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/spatial_autocorrelation/adata_all_techniques.h5ad')
old_regions = olddata.obs['region_smooth_subsampled'].unique()
palette = sc.pl.palettes.default_102
old_cmap = {old_regions[i]:palette[i] for i in range(len(old_regions))}

In [ ]:
old_cmap['L6b'] = sc.pl.palettes.default_102[7]

In [ ]:
subdata = adata[adata.obs['species']=='marmoset']
sc.set_figure_params(figsize=(2.4,6))
sc.pl.embedding(subdata, basis='spatial', color='region', palette=old_cmap, s=5, return_fig=True, frameon=False, title='', edgecolor=None, legend_loc=None)
figname = f'tissue_marmoset_colored_by_region.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
subdata = adata[adata.obs['species']=='mouse']

plt.figure(figsize=(5,6))
plt.scatter(*adata_full.obsm['spatial'].T, color=[0.7]*3, s=30)
plt.scatter(*subdata.obsm['spatial'].T, c=[old_cmap[region] for region in subdata.obs['region']], s=30)
plt.axis('off')

# sc.set_figure_params(figsize=(5,6))
# sc.pl.embedding(subdata, basis='spatial', color='region', palette=old_cmap, s=80, frameon=False, title='', edgecolor=None, ax=plt.gca(), legend_loc=None)
figname = f'tissue_mouse_colored_by_region.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
species = ['marmoset', 'mouse']
for i in range(len(species)):
    subdata = adata[adata.obs['species']==species[i]]
    sc.set_figure_params(figsize=(5,5))
    sc.pl.embedding(subdata, basis='X_umap_spin', color='region', palette=old_cmap, s=10, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
    plt.show()

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='region', palette=old_cmap, s=10, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
figname = f'latent_colored_by_region_xspecies.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='species', s=20, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
figname = f'latent_colored_by_species.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

# Markers

## L2/3, shared

In [ ]:
region = 'L2/3'

In [ ]:
adata.layers['scaled'] = adata.X.copy()

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='region')
marker_df = sc.get.rank_genes_groups_df(adata, group=region)
shared_marker = marker_df['names'][0]

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=old_cmap['L2/3'], linewidth=1, edgecolor='k')
plt.grid(None)
plt.axis('off')
figname = f'region_deg_score_hist_xspecies.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=old_cmap['L2/3'], linewidth=1, edgecolor='k')
plt.grid(None)
plt.show()

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names'][:10]

In [ ]:
marker_df['scores'][5]

In [ ]:
xdata = adata[adata.obs['region']==region]
sc.tl.rank_genes_groups(xdata, groupby='species')
marker_df = sc.get.rank_genes_groups_df(xdata, group='mouse')
mouse_markers = marker_df['names']
mouse_marker = mouse_markers[0]

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=old_cmap['L2/3'], linewidth=1, edgecolor='k')
plt.grid(None)
plt.axis('off')
figname = f'L23_deg_score_hist_xspecies.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=old_cmap['L2/3'], linewidth=1, edgecolor='k')
plt.grid(None)
plt.show()

### Zooms

In [ ]:
def get_zoom(adata, x, y, width, height):
    zoomdata = adata[adata.obsm['spatial'][:,0]>x]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,0]<x+width]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]>y]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]<y+height]
    return zoomdata

In [ ]:
# marmoset_box = (-4400, 10800, 1400, 4200)
marmoset_box = (2200, 10200, 1600, 4800)
subdata = adata[adata.obs['species']=='marmoset']

plt.figure(figsize=(1.5,4))
plt.scatter(*subdata.obsm['spatial'].T, s=1)
rect = Rectangle(marmoset_box[:2], marmoset_box[2], marmoset_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
# mouse_box = (2600, 8200, 2050, 6150)
mouse_box = (5300, -3650, 700, 2100)
subdata = adata[adata.obs['species']=='mouse']

plt.figure(figsize=(4,4))
plt.scatter(*subdata.obsm['spatial'].T, s=20, c=[old_cmap[region] for region in subdata.obs['region']])
rect = Rectangle(mouse_box[:2], mouse_box[2], mouse_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
# ctxdata = adata[adata.obs['region']!='meninges/L1']
# ctxdata = ctxdata[ctxdata.obs['region']!='fiber tract']
ctxdata = adata

In [ ]:
zoomdata_marmoset = get_zoom(ctxdata[ctxdata.obs['species']=='marmoset'], *marmoset_box)
zoomdata_mouse = get_zoom(ctxdata[ctxdata.obs['species']=='mouse'], *mouse_box)

In [ ]:
linewidths = [0.05, 0.5]

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
ss = [80, 2000]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color='region', palette=old_cmap, s=ss[i], return_fig=True, frameon=False, title='', legend_loc=None, edgecolor='k', linewidth=linewidths[i])
    figname = f'zoom_tissue_{species[i]}_colored_by_regions.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
# X = sc.concat((zoomdata_marmoset, zoomdata_mouse))[:,[shared_marker]].X
# vmax = X.max()
# vmin = X.min()
cmaps = ['Blues', 'Oranges']

In [ ]:
subdata_marmoset = ctxdata[ctxdata.obs['species']=='marmoset']
subdata_mouse = ctxdata[ctxdata.obs['species']=='mouse']

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
subdatas = [subdata_marmoset, subdata_mouse]
ss = [50, 200]
for i in range(len(species)):
    sc.pl.embedding(subdatas[i], basis='spatial', color=shared_marker, s=ss[i], cmap=cmaps[i],
    return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor='k', linewidth=0.05,
    )# vmin=vmin, vmax=vmax)
    # sc.pl.embedding(subdatas[i], basis='spatial', color=shared_marker, s=100, cmap='inferno_r', return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    # figname = f'zoom_tissue_{species[i]}_colored_by_{shared_marker}.png'
    # savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
subdatas = [subdata_marmoset, subdata_mouse]
ss = [50, 200]
for i in range(len(species)):
    sc.pl.embedding(subdatas[i], basis='spatial', color=mouse_marker, s=ss[i], cmap=cmaps[i],
    return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor='k', linewidth=0.05,
    )# vmin=vmin, vmax=vmax)
    # sc.pl.embedding(subdatas[i], basis='spatial', color=mouse_marker, s=100, cmap='inferno_r', return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    # figname = f'zoom_tissue_{species[i]}_colored_by_{mouse_marker}.png'
    # savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
ss = [80, 2000]

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=shared_marker, s=ss[i], cmap=cmaps[i],
    return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor='k', linewidth=linewidths[i],
    )# vmin=vmin, vmax=vmax)
    # sc.pl.embedding(zoomdatas[i], basis='spatial', color=shared_marker, s=100, cmap='inferno_r', return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    figname = f'zoom_tissue_{species[i]}_colored_by_{shared_marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=mouse_marker, s=ss[i], cmap=cmaps[i],
    return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor='k', linewidth=linewidths[i],
    )# vmin=vmin, vmax=vmax)
    # sc.pl.embedding(zoomdatas[i], basis='spatial', color=mouse_marker, s=100, cmap='inferno_r', return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    figname = f'zoom_tissue_{species[i]}_colored_by_{mouse_marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

## L2/3, unique

### STARmap PLUS

In [ ]:
xdata = pooldata[pooldata.obs['region']==region]
sc.tl.rank_genes_groups(xdata, groupby='tech')
marker_df = sc.get.rank_genes_groups_df(xdata, group='starmap')
starmap_markers = marker_df['names']
starmap_marker = visium_markers[0]

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
marker = marker_df['names'][0]
print(marker)
for i in range(len(techs)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = pooldata[pooldata.obs['tech']==techs[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[techs[i]], title='', colorbar_loc=None, return_fig=True, layer='scaled')
    plt.axis('off')
    figname = f'{techs[i]}_tissue_colored_by_{marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

### Slide-seqV2

In [ ]:
xdata = pooldata[pooldata.obs['region']==region]
sc.tl.rank_genes_groups(xdata, groupby='tech')
marker_df = sc.get.rank_genes_groups_df(xdata, group='slideseq')
slideseq_markers = marker_df['names']
slideseq_marker = slideseq_markers[0]

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
marker = marker_df['names'][0]
print(marker)
for i in range(len(techs)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = pooldata[pooldata.obs['tech']==techs[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[techs[i]], title='', colorbar_loc=None, return_fig=True, layer='scaled')
    plt.axis('off')
    figname = f'{techs[i]}_tissue_colored_by_{marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

### Visium

In [ ]:
xdata = pooldata[pooldata.obs['region']==region]
sc.tl.rank_genes_groups(xdata, groupby='tech')
marker_df = sc.get.rank_genes_groups_df(xdata, group='visium')
visium_markers = marker_df['names']
visium_marker = visium_markers[0]

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
marker = marker_df['names'][520]
print(marker)
for i in range(len(techs)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = pooldata[pooldata.obs['tech']==techs[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[techs[i]], title='', colorbar_loc=None, return_fig=True)
    plt.axis('off')
    figname = f'{techs[i]}_tissue_colored_by_{marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

## Thalamus

In [ ]:
adata_starmap = pooldata[pooldata.obs['tech']=='starmap']
adata_slideseq = pooldata[pooldata.obs['tech']=='slideseq']
adata_visium = pooldata[pooldata.obs['tech']=='visium']
adatas = [adata_starmap, adata_slideseq, adata_visium]

rawdatas = [sc.read_h5ad(f'/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xtech/adata_{tech}.h5ad') for tech in techs]

In [ ]:
genes_starmap = set(rawdatas[0].var_names)
genes_slideseq = set(rawdatas[1].var_names)
genes_visium = set(rawdatas[2].var_names)

unique_genes_starmap = list(genes_starmap - (genes_slideseq|genes_visium))
udata_starmap = rawdatas[0][:,unique_genes_starmap]
udata_starmap.obs['region'] = adata_starmap.obs['region']

unique_genes_slideseq = list(genes_slideseq - (genes_starmap|genes_visium))
udata_slideseq = rawdatas[1][:,unique_genes_slideseq]
udata_slideseq.obs['region'] = adata_slideseq.obs['region']
del udata_slideseq.uns['log1p']

unique_genes_visium = list(genes_visium - (genes_slideseq|genes_starmap))
udata_visium = rawdatas[2][:,unique_genes_visium]
udata_visium.obs['region'] = adata_visium.obs['region']
del udata_visium.uns['log1p']

udatas = [udata_starmap, udata_slideseq, udata_visium]

In [ ]:
# Shared
num_markers = 5
unique_markers = dict()
for i in range(len(udatas)):
    udata = udatas[i]
    sc.tl.rank_genes_groups(udata, groupby='region', use_raw=False)
    marker_df = sc.get.rank_genes_groups_df(udata, group='0')
    unique_markers[techs[i]] = marker_df['names'][:num_markers]

In [ ]:
for i in range(len(techs)):
    sc.set_figure_params(figsize=figsizes[i])
    unique_marker = unique_markers[techs[i]][0]
    print(unique_marker)
    sc.pl.embedding(udatas[i], basis='spatial', color=unique_marker, s=ss[techs[i]], cmap='inferno_r', return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    figname = f'tissue_{techs[i]}_colored_by_{unique_marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
fig = plt.figure()
cax = plt.imshow(np.array([[0,1]]), cmap='plasma_r')
plt.gca().set_visible(False)
cbar = fig.colorbar(cax, ticks=[0, 1])
cbar.ax.set_yticklabels(['', ''])
figname = 'cbar.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight')
plt.show()

## Troubleshooting

In [ ]:
region = '0'

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='region')
marker_df = sc.get.rank_genes_groups_df(adata, group=region)
shared_marker = marker_df['names'][0]

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
marker = marker_df['names'][0]
print(marker)
for i in range(len(species)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = adata[adata.obs['species']==species[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[i], title='', colorbar_loc=None, return_fig=True)
    plt.axis('off')

In [ ]:
marker = marker_df['names'][1]
print(marker)
for i in range(len(species)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = adata[adata.obs['species']==species[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[i], title='', colorbar_loc=None, return_fig=True)
    plt.axis('off')

In [ ]:
marker = marker_df['names'][2]
print(marker)
for i in range(len(species)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = adata[adata.obs['species']==species[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[i], title='', colorbar_loc=None, return_fig=True)
    plt.axis('off')

In [ ]:
marker = marker_df['names'][3]
print(marker)
for i in range(len(species)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = adata[adata.obs['species']==species[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[i], title='', colorbar_loc=None, return_fig=True)
    plt.axis('off')

In [ ]:
marker = marker_df['names'][4]
print(marker)
for i in range(len(species)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = adata[adata.obs['species']==species[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[i], title='', colorbar_loc=None, return_fig=True)
    plt.axis('off')

In [ ]:
marker = marker_df['names'][5]
print(marker)
for i in range(len(species)):
    sc.set_figure_params(figsize=figsizes[i])
    subdata = adata[adata.obs['species']==species[i]]
    sc.pl.embedding(subdata, basis='spatial', color=marker, cmap='plasma_r', s=ss[i], title='', colorbar_loc=None, return_fig=True)
    plt.axis('off')